In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd "/content/drive/MyDrive/COMP598-datascience"

/content/drive/MyDrive/COMP598-datascience


In [4]:
import csv
import pandas as pd

In [5]:
ds = pd.read_csv("IRAhandle_tweets_1.csv")

In [6]:
ds.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [7]:
ds.shape

(243891, 21)

In [8]:
ds = ds[:10000]

In [9]:
ds.shape

(10000, 21)

In [10]:
ds['language'].unique()

array(['English', 'Russian', 'Serbian', 'Ukrainian', 'Tagalog (Filipino)',
       'Albanian', 'Italian', 'Romanian', 'Spanish', 'Catalan', 'German',
       'Estonian', 'French', 'Norwegian', 'Vietnamese', 'Dutch', 'Arabic',
       'Uzbek', 'Bulgarian', 'Macedonian', 'Farsi (Persian)', 'Turkish',
       'LANGUAGE UNDEFINED', 'Czech', 'Somali', 'Lithuanian', 'Croatian',
       'Slovak', 'Icelandic'], dtype=object)

In [11]:
## now we know there is only one spelling of "English" (and not "ENGLISH" or "english" etc)

In [12]:
import re

In [13]:
temp_df = []
for idx in range(ds.shape[0]):
    ## if english and if no question mark
    if ds['language'][idx] == 'English' and re.search(r'\?', ds['content'][idx]) == None:
        temp_df.append(list(ds.iloc[idx]))
          
    

In [14]:
df = pd.DataFrame(temp_df, columns=ds.columns)

In [15]:
df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [16]:
df.shape

(4802, 21)

In [17]:
df['post_type'].unique()

array([nan, 'RETWEET', 'QUOTE_TWEET'], dtype=object)

In [18]:
## WE HAVE MORE THAN WHAT WE WANT BECAUSE SOME ARE RETWEETS!!!!!!!!!!!
## Some people do not capitalise T
## Some people reply and do not mention trump


In [19]:
pattern = r'(^(Trump)|\W(Trump))\W' # string starts with 'Trump', or has not an alphanumeric followed by Trump, and is not followed by an alphanumeric
trump_mention = []

for idx in range(df.shape[0]):
    if re.search(pattern, df['content'][idx]):
        trump_mention.append("Y")
    else:
        trump_mention.append("N")
        
df['trump_mention'] = trump_mention

In [32]:
## IDENTIFYING REALDONALDTRUMP MENTION 

pattern = r'\@(realDonaldTrump)' # string containing the mention @realDonaldTrump
dtrump_mention = []

for idx in range(df.shape[0]):
    if re.search(pattern, df['content'][idx]):
        dtrump_mention.append("Y")
    else:
        dtrump_mention.append("N")
        
df['dtrump_mention'] = dtrump_mention

In [33]:
df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1,trump_mention,dtrump_mention
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN,N,N
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN,Y,N
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN,N,N
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN,Y,N
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN,N,N


In [30]:
##### 879 retweet or quote tweet
##### 590 retweet
##### 289 quote tweet

In [38]:
## HERE WE COUNT IF REALDONALDTRUMP IS MENTIONED

rt_count = 0
qt_count = 0
count = 0
for idx in range(df.shape[0]):
    if df['dtrump_mention'][idx] == "Y":
        if df['post_type'][idx] == 'RETWEET':
            rt_count += 1
        elif df['post_type'][idx] == 'QUOTE_TWEET':
            qt_count += 1
        else:
            count += 1

print("Normal: {}\nRetweet: {}\nQuote tweet: {}\nTotal: {}".format(count, rt_count, qt_count, count+rt_count+qt_count))

Normal: 72
Retweet: 5
Quote tweet: 0
Total: 77


In [34]:
## HERE WE COUNT IF REALDONALDTRUMP IS MENTIONED AND IF POST DOES NOT INCLUDE WORD "Trump"

rt_count = 0
qt_count = 0
count = 0
for idx in range(df.shape[0]):
    if df['dtrump_mention'][idx] == "Y" and df['trump_mention'][idx] == "N":
        if df['post_type'][idx] == 'RETWEET':
            rt_count += 1
        elif df['post_type'][idx] == 'QUOTE_TWEET':
            qt_count += 1
        else:
            count += 1

print("Normal: {}\nRetweet: {}\nQuote tweet: {}\nTotal: {}".format(count, rt_count, qt_count, count+rt_count+qt_count))

Normal: 68
Retweet: 3
Quote tweet: 0
Total: 71


In [40]:
## HERE WE COUNT IN HOW MANY POSTS "Trump" was mentioned

rt_count = 0
qt_count = 0
count = 0
for idx in range(df.shape[0]):
    if df['trump_mention'][idx] == "Y":
        if df['post_type'][idx] == 'RETWEET':
            rt_count += 1
        elif df['post_type'][idx] == 'QUOTE_TWEET':
            qt_count += 1
        else:
            count += 1
trump_mentions = count+rt_count+qt_count
print("Normal: {}\nRetweet: {}\nQuote tweet: {}\nTotal: {}".format(count, rt_count, qt_count, trump_mentions))

Normal: 77
Retweet: 30
Quote tweet: 1
Total: 108


In [41]:
final_df = pd.DataFrame(data= {'tweet_id': df['tweet_id'], 'publish_date': df['publish_date'], 'content': df['content'], 'trump_mention': df['trump_mention']})

In [42]:
final_df.head()

,tweet_id,publish_date,content,trump_mention
0,914580356430536707,10/1/2017 19:58,"""We have a sitting Democrat US Senator on tria...",N
1,914621840496189440,10/1/2017 22:43,Marshawn Lynch arrives to game in anti-Trump s...,Y
2,914623490375979008,10/1/2017 22:50,Daughter of fallen Navy Sailor delivers powerf...,N
3,914639143690555392,10/1/2017 23:52,JUST IN: President Trump dedicates Presidents ...,Y
4,914312219952861184,10/1/2017 2:13,"19,000 RESPECTING our National Anthem! #StandF...",N


In [43]:
## tweet_id,publish_date,content,and trump_mention

In [44]:
final_df.head()

,tweet_id,publish_date,content,trump_mention
0,914580356430536707,10/1/2017 19:58,"""We have a sitting Democrat US Senator on tria...",N
1,914621840496189440,10/1/2017 22:43,Marshawn Lynch arrives to game in anti-Trump s...,Y
2,914623490375979008,10/1/2017 22:50,Daughter of fallen Navy Sailor delivers powerf...,N
3,914639143690555392,10/1/2017 23:52,JUST IN: President Trump dedicates Presidents ...,Y
4,914312219952861184,10/1/2017 2:13,"19,000 RESPECTING our National Anthem! #StandF...",N


In [45]:
final_df.to_csv('/content/drive/MyDrive/COMP598-datascience/dataset.tsv', sep='\t', index_label=False)

In [46]:
frac = trump_mentions / (final_df.shape[0])
print(frac)
print(round(frac, 3))

0.022490628904623073
0.022


In [47]:
results_df = pd.DataFrame(data= {'result': ['frac-trump-mentions'], 'value': [round(frac, 3)]})

In [48]:
results_df

,result,value
0,frac-trump-mentions,0.022


In [36]:
results_df.to_csv('/content/drive/MyDrive/COMP598-datascience/results.tsv', sep='\t', index=False)